In [1]:
import numpy as np
import os
from scipy.stats import linregress
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import h5py

#### 1) Separate the ocean into 45 regions based on the basin code and neutral density values

- Using "basinmask_01.msk" file from World Ocean Atlas available here: https://www.ncei.noaa.gov/products/world-ocean-atlas

- Basin codes are already pre-processed and encoded such that

  1: Atlantic,

  2: Pacific,

  3: Indian,

  4: Southern Ocean,

  5: Arctic


- Neutral density $\gamma(T, S, x, y, p)$ is calculated using the code from https://www.teos-10.org/preteos10_software/neutral_density.html based on Jackett, David R., Trevor J. McDougall, 1997: A Neutral Density Variable for the World's Oceans. J. Phys. Oceanogr., 27, 237–263. doi: 10.1175/1520-0485(1997)0272.0.CO;2

&emsp; &emsp; &emsp; * $T(x,y,z)$ and $S(x,y,z)$ are available from World Ocean Atlas. 

&emsp; &emsp; &emsp; &emsp; &emsp; &emsp; Specifically, we used the $1991-2020$ climate normal because that most corresponds to the time period in ECCO-Darwin.

&emsp; &emsp; &emsp; * Pressure $p$ is calculated from latitude and depth, so $p = p(x,z)$ using Gibbs-Seawater Oceanographic Toolbox (https://www.teos-10.org/software.htm).

&emsp; &emsp; &emsp; * $\gamma(T, S, x, y, p)$ is interpolated onto the ECCO-Darwin grid ($1^\circ$ horizontal resolution) from the WOA grid.

In [15]:
# Load ocean basin codes (1: Atlantic, 2: Pacific, 3: Indian, 4: Southern Ocean, 5: Arctic)
with h5py.File('../processed_data/basin_codes.h5', 'r') as f:
    basin_code = f['basin_code'][:]

# Load gamma: 3D array (180, 360, 50)
with h5py.File('../processed_data/gamma.h5', 'r') as f:
    gamma = f['gamma'][:]

# Load latitude/longitude/depth as 3D arrays (180, 360, 50)
with h5py.File('../processed_data/grid.h5', 'r') as f:
    lat = f['latitude'][:]
    lon = f['longitude'][:]
    depth = f['depth'][:]

In [13]:
# gamma cutoff ranges for Atlantic, Pacific, Indian Oceans
gamma_cutoff_API = [0, 26.00, 26.50, 26.75, 27.00, 27.25, 27.50, 27.75, 27.85, 27.95, 28.05, 30] 

# gamma cutoff ranges for Southern Ocean
gamma_cutoff_SO = [0, 27.00, 27.25, 27.50, 27.75, 27.85, 27.95, 28.05, 28.10, 28.15, 28.20, 30] 

In [17]:
# Initialize the dataframe
df_regions = pd.DataFrame(columns=['latitude', 'longitude', 'depth', 'basin_code', 'gamma'])

# We need to reshape all the 3D arrays into 1D arrays to add as columns into the dataframes
# latitude
df_regions['latitude'] = lat.reshape((180*360*50,))
# longitude
df_regions['longitude'] = lon.reshape((180*360*50,))
# depth (need to get adjusted for big/little endian - don't worry about understanding this)
df_regions['depth'] = depth.reshape((180*360*50,)).byteswap().newbyteorder('=')
# basin codes (Atlantic/Pacific/Indian/Southern/Arctic) --> store 
df_regions['basin_code'] = basin_code.reshape((180*360*50,)).astype(int)
# neutral density \gamma(x,y,z)
df_regions['gamma'] = gamma.reshape((180*360*50,))

In [19]:
# initialize the "region code" column with -100 (so land areas will be -100 after we are done assigning codes to the ocean)

df_regions['region_code'] = -100

# start the "region code" counter (first region will be 0, and then so on) 
region_counter = 0 

print("region_counter", "basin_code", "gamma range")

# loop over 4 basins: Atlantic, Pacific, Indian, Southern Oceans (Artic Ocean will just get its own code)
for ind_basin in range(4):
    if ind_basin<3:
        gamma_cutoff = gamma_cutoff_API
    else:
        gamma_cutoff = gamma_cutoff_SO
    # loop over the gamma ranges
    for ind_gamma in range(len(gamma_cutoff)-1):
        # look for rows within the dataframe where:
        #        "basin_code" is the given basin (Atlantic = 1, Pacific = 2, Indian = 3, Southern = 4)
        #        "gamma" is within the given range 
        ind_list = list(df_regions[(df_regions['basin_code'] ==ind_basin+1) & 
                            (df_regions['gamma']>=gamma_cutoff[ind_gamma]) & (df_regions['gamma']<gamma_cutoff[ind_gamma+1])].index)
        
        # print the region if there are fewer than 2000 grid points in it:
        if len(ind_list)<2000:
            print(region_counter, ind_basin+1, [gamma_cutoff[ind_gamma], gamma_cutoff[ind_gamma+1]])

        # for rows that match these conditions, assign the "region code"
        df_regions['region_code'].iloc[ind_list] = region_counter
        
        # update the region code counter with a new value
        region_counter += 1

# Find all rows for the Arctic Ocean and assign its own region code
ind_list = list(df_regions[(df_regions['basin_code'] ==5)].index)
df_regions['region_code'].iloc[ind_list] = region_counter

region_counter basin_code gamma range


Now we don't have any regions with grid points less than 2000!

Let's see how many regions we have now:

In [22]:
print('Total number of regions: ', region_counter)

Total number of regions:  44


In [26]:
# Save dataset as HDF5
with h5py.File('../processed_data/basin_codes.h5', 'w') as f:
    dset = f.create_dataset('basin_code', data=basin_code)
    f.create_dataset('region_code', data=df_regions['region_code'].to_numpy().reshape((180,360,50)))
    f.create_dataset('gamma', data=gamma)
    f['gamma'].attrs['unit'] = 'kg/m^3'

#### 2) Area and volume of each grid cell

Area of each grid cell is equal to:
$$
\begin{equation}
A(x,y) = \pi R^2 |\sin{\phi_2} - \sin{\phi_1}| \left(\frac{\lambda_2-\lambda_1}{180}\right),
\end{equation}
$$
where $R=6371\times 10^{3}$ m (which is the radius of the Earth), $\phi_2$ and $\phi_1$ are latitudinal bounds of a cell (in radians), and $\lambda_2$ and $\lambda_1$ are longitudinal bounds of a cell (in radians). 

This is why we define latitude and longitude in terms of cell centers. For example, a cell with latitude and longitude coordinates $(x,y)=(-89.5, 0)$ will have bounds for a grid with one degree resolution: $\phi_2 = x+0.5 = -89$, $\phi_1 = x-0.5 = -90$, $\lambda_2=y+0.5=0.5$, $\lambda_1 = y-0.5 = -0.5$. Also, for a one degree resolution, we will always have $\lambda_2-\lambda_1=1$. So, the area will be only a function of latitude.

In [31]:
# 1-degree resolution ECCO-Darwin data: want cell centers

lat_interp = np.arange(-89.5, 90.5, 1)
lon_interp = np.arange(-179.5, 180.5, 1)

In [33]:
area = np.zeros((180,360))
R = 6371.e3
# convert degrees to radians
lon_diff = np.pi/180

for indx in range(len(lat_interp)):
    phi2 = (lat_interp[indx]+0.5)*np.pi/180
    phi1 = (lat_interp[indx]-0.5)*np.pi/180
    area[indx, :] = lon_diff*(R**2)*abs(np.sin(phi2)-np.sin(phi1))

Now we need volume of each grid cell. The volume ($V$) is area of the cell ($A$) multiplied by thickness of the cell ($\Delta z$):
$$
\begin{equation}
V = A*\Delta z
\end{equation}
$$
Thickness of each cell is available from the 'DRF.data' file from ECCO-Darwin model. It returns a 1D array (50-cells long) with thickness for each depth layer.

In [36]:
# Load \Delta z (vertical grid spacing)
drf_path = "../processed_data/DRF.data" 
drf = np.fromfile(drf_path, dtype='>f4')[:50]

In [40]:
volume_3D = np.zeros((180, 360, 50))
area_3D = np.zeros((180, 360, 50))

for indz in range(len(drf)):
    area_3D[:,:,indz] = area
    volume_3D[:,:,indz] = area*drf[indz]

In [42]:
# Save data as HDF5 file
with h5py.File('../processed_data/area_volume.h5', 'w') as f:
    dset = f.create_dataset('Area', data=area_3D)
    f.create_dataset('Volume', data=volume_3D)
    f['Area'].attrs['unit'] = 'm^2'
    f['Volume'].attrs['unit'] = 'm^3'